## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Vaini,-21.2000,-175.2000,86.16,74,40,5.75,TO,2022-02-03 04:22:51,scattered clouds
1,1,Ryotsu,38.0833,138.4333,37.36,76,100,10.31,JP,2022-02-03 04:24:25,overcast clouds
2,2,Zwedru,6.0667,-8.1281,68.68,99,41,2.24,LR,2022-02-03 04:24:26,scattered clouds
3,3,Hastings,50.8552,0.5729,48.00,88,100,15.37,GB,2022-02-03 04:24:26,overcast clouds
4,4,Busselton,-33.6500,115.3333,90.05,31,87,8.79,AU,2022-02-03 04:24:26,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
7,7,Kavaratti,10.5669,72.6420,79.93,70,12,8.08,IN,2022-02-03 04:24:27,very heavy rain
9,9,Puerto Ayora,-0.7393,-90.3518,74.55,93,1,3.00,EC,2022-02-03 04:22:53,clear sky
10,10,Port Alfred,-33.5906,26.8910,73.04,91,90,12.35,ZA,2022-02-03 04:24:28,overcast clouds
14,14,Nata,8.3333,-80.5167,72.59,77,28,5.88,PA,2022-02-03 04:24:29,scattered clouds
23,23,Hithadhoo,-0.6000,73.0833,81.59,70,6,4.07,MV,2022-02-03 04:24:31,very heavy rain
25,25,Duverge,18.3784,-71.5247,70.90,88,5,0.25,DO,2022-02-03 04:24:32,clear sky
27,27,Indramayu,-6.3264,108.3200,83.62,72,95,9.80,ID,2022-02-03 04:24:32,light rain
28,28,Catamarca,-28.4696,-65.7852,78.78,68,12,2.46,AR,2022-02-03 04:24:33,few clouds
32,32,Hilo,19.7297,-155.0900,73.40,79,100,4.61,US,2022-02-03 04:22:55,overcast clouds
39,39,Catuday,16.2923,119.8062,82.06,60,5,6.04,PH,2022-02-03 04:24:35,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kavaratti,IN,79.93,very heavy rain,10.5669,72.6420,
9,Puerto Ayora,EC,74.55,clear sky,-0.7393,-90.3518,
10,Port Alfred,ZA,73.04,overcast clouds,-33.5906,26.8910,
14,Nata,PA,72.59,scattered clouds,8.3333,-80.5167,
23,Hithadhoo,MV,81.59,very heavy rain,-0.6000,73.0833,
25,Duverge,DO,70.90,clear sky,18.3784,-71.5247,
27,Indramayu,ID,83.62,light rain,-6.3264,108.3200,
28,Catamarca,AR,78.78,few clouds,-28.4696,-65.7852,
32,Hilo,US,73.40,overcast clouds,19.7297,-155.0900,
39,Catuday,PH,82.06,clear sky,16.2923,119.8062,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(100)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Kavaratti,IN,79.93,very heavy rain,10.5669,72.6420,Hotels in Lakshadweep Islands
9,Puerto Ayora,EC,74.55,clear sky,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Port Alfred,ZA,73.04,overcast clouds,-33.5906,26.8910,The Halyards Hotel
14,Nata,PA,72.59,scattered clouds,8.3333,-80.5167,Rey David
23,Hithadhoo,MV,81.59,very heavy rain,-0.6000,73.0833,Scoop Guest House
...,...,...,...,...,...,...,...
282,Omboue,GA,77.41,overcast clouds,-1.5746,9.2618,Hotel Olako
286,Lorengau,PG,79.18,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
287,Kupang,ID,82.35,moderate rain,-10.1667,123.5833,Hotel On The Rock by Prasanthi
290,Alice Springs,AU,76.55,overcast clouds,-23.7000,133.8833,Desert Palms Alice Springs


In [25]:
hotel_df.isna().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [26]:
# 7. Drop the rows where there is no Hotel Name.


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))